In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
TRAIN_DIR = '/kaggle/input/face-expression-recognition-dataset/images/train'

emotion_classes = [d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]

print(f'Emotion Classes:', emotion_classes)


In [ ]:
emotion_classes = ['surprise', 'fear', 'angry', 'neutral', 'sad', 'disgust', 'happy']

# Data Augmentation

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=15,
    #shear_range=0.0,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale = 1./255)

# Data Generators

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/face-expression-recognition-dataset/images/train',
    target_size = (48, 48),
    color_mode = 'grayscale',  # If rgb use this
    batch_size = 32,
    shuffle=True,
    classes=emotion_classes,
    class_mode= 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/kaggle/input/face-expression-recognition-dataset/images/validation',
    target_size = (48, 48),
    color_mode = 'grayscale',  # If rgb use this
    batch_size = 32,
    shuffle=False,
    classes=emotion_classes,
    class_mode= 'categorical'
)

In [ ]:
# If you use preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input

IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [ ]:
class_names = list(train_generator.class_indices.keys())
NUM_CLASSES = train_generator.num_classes

print("Classes:", class_names)
print("Number of classes:", NUM_CLASSES)

In [ ]:
print(train_generator.class_indices)


In [ ]:
# Visualize Sample Images

for images, labels in train_generator:
    plt.figure(figsize=(10, 5))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        img = images[i]
        plt.imshow(img, cmap = 'gray')  # if rgb no cmap needed

        true_index = np.argmax(labels[i])
        true_label = class_names[true_index]
        plt.title(true_label)
        plt.axis('off')
    break

plt.show()

In [ ]:
# IF the image is RGB then you can use transfer learning
# Build VGG16 Emotion Model

# base_model = VGG16(
#     weights="imagenet",
#     include_top=False,
#     input_shape=(IMG_SIZE, IMG_SIZE, 3)
# )

# # Freeze base model
# base_model.trainable = False

# x = base_model.output
# x = Flatten()(x)
# x = Dense(256, activation="relu")(x)
# x = Dropout(0.5)(x)

# outputs = Dense(NUM_CLASSES, activation="softmax")(x)

# model = Model(inputs=base_model.input, outputs=outputs)

# model.compile(
#     optimizer="adam",
#     loss="categorical_crossentropy",
#     metrics=["accuracy"]
# )

# model.summary()


In [ ]:
# Build CNN Model

model = Sequential([

    Conv2D(32, kernel_size= (3,3), activation='relu', input_shape = (48, 48, 1)),  # if rgb - then inputshape = (48,48, 3)
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, kernel_size= (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, kernel_size= (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    # Dense Layers
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation="softmax")
])

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
# Train the Model

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15
)

# history = cnn_model.fit(
#     train_generator,
#     validation_data=val_generator,
#     epochs=25
# )



In [ ]:
# Plot Accuracy & Loss

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(acc, label="Train Accuracy")
plt.plot(val_acc, label="Val Accuracy")
plt.title("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(loss, label="Train Loss")
plt.plot(val_loss, label="Val Loss")
plt.title("Loss")
plt.legend()

plt.show()


In [ ]:
# Evaluate on Test Set

# test_loss, test_acc = model.evaluate(test_generator)
# print(f"Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
# Predict ONE random test image (Correct for Emotion)

#for images, labels in test_generator:
for images, labels in val_generator:
    img = images[0]

    # True label (categorical)
    true_index = np.argmax(labels[0])      # ✅ one-hot → class index
    true_label = class_names[true_index]

    # Prediction
    preds = model.predict(np.expand_dims(img, axis=0), verbose=0)[0]
    # preds shape = (NUM_CLASSES,)

    pred_index = np.argmax(preds)          # ✅ softmax → class index
    pred_label = class_names[pred_index]
    confidence = preds[pred_index] * 100

    # Show
    plt.imshow(img)
    plt.title(
        f"Actual: {true_label}\n"
        f"Predicted: {pred_label}\n"
        f"Confidence: {confidence:.2f}%"
    )
    plt.axis("off")
    break


In [ ]:
# Predict Random Test Images

#for images, labels in test_generator:
for images, labels in val_generator:
    plt.figure(figsize=(8,8))
    for i in range(9):
        img = images[i]
        true_index = np.argmax(labels[i])  # categorical label
        true_label = class_names[true_index]

        preds = model.predict(np.expand_dims(img, axis=0), verbose=0)[0]  # preds shape = (NUM_CLASSES,)
        pred_index = np.argmax(preds)
        pred_label = class_names[pred_index]
        confidence = preds[pred_index] * 100

        plt.subplot(3,3,i+1)
        plt.imshow(img, cmap = 'gray') # if rgb no gray
        plt.title(f"A: {true_label}\nP: {pred_label}\n{confidence:.2f}%")
        plt.axis("off")
    break
plt.show()


In [ ]:
# Save Model

model.save("emotion_vgg16_model.h5")
print("✅ Model saved")


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

img_path = "/content/test_emotion.jpg"   # change path

# Load & preprocess
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE), color_mode="grayscale") # grey scale
# img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE)) # color mode rgb
img_array = image.img_to_array(img) / 255.0  # shape = (H, W, 3)
img_array = np.expand_dims(img_array, axis=0) # shape = (1, H, W, 3)

# Predict
preds = model.predict(img_array, verbose=0)[0]

pred_index = np.argmax(preds)
predicted_label = class_names[pred_index]
confidence = preds[pred_index] * 100

# Show result
plt.imshow(img, cmap='gray') # for grey otherwise not needed grey if rgb images is there
plt.axis("off")
plt.title(f"Predicted: {predicted_label}\nConfidence: {confidence:.2f}%")
plt.show()


In [ ]:
# CASE 1 — Using VGG16 Model (RGB, 224×224)

import cv2
import numpy as np
import matplotlib.pyplot as plt

img_path = "/content/test_emotion.jpg"   # change path

# Load image using cv2 (BGR)
img_bgr = cv2.imread(img_path)

# Resize
img_bgr = cv2.resize(img_bgr, (IMG_SIZE, IMG_SIZE))

# Convert BGR -> RGB
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

# Normalize
img_array = img_rgb.astype("float32") / 255.0

# Add batch dimension
img_array = np.expand_dims(img_array, axis=0)

# Predict
preds = model.predict(img_array, verbose=0)[0]

pred_index = np.argmax(preds)
predicted_label = class_names[pred_index]
confidence = preds[pred_index] * 100

# Show result
plt.imshow(img_rgb)
plt.axis("off")
plt.title(f"Predicted: {predicted_label}\nConfidence: {confidence:.2f}%")
plt.show()



In [ ]:
# Using preprocess_input in ImageDataGenerator

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input

img_path = "/content/test_emotion.jpg"   # change path

# Load image using cv2 (BGR)
img_bgr = cv2.imread(img_path)

# Resize
img_bgr = cv2.resize(img_bgr, (IMG_SIZE, IMG_SIZE))

# Convert BGR -> RGB (optional but good for display)
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

# Add batch dimension first
img_array = np.expand_dims(img_rgb, axis=0)

# 🔥 VGG16 preprocessing (REPLACES /255 normalization)
img_array = preprocess_input(img_array)

# Predict
preds = model.predict(img_array, verbose=0)[0]

pred_index = np.argmax(preds)
predicted_label = class_names[pred_index]
confidence = preds[pred_index] * 100

# Show result
plt.imshow(img_rgb)
plt.axis("off")
plt.title(f"Predicted: {predicted_label}\nConfidence: {confidence:.2f}%")
plt.show()


In [ ]:
# CASE 2 — Using Custom CNN with Grayscale (48×48×1)

import cv2
import numpy as np
import matplotlib.pyplot as plt

img_path = "/content/test_emotion.jpg"   # change path

# Load image in grayscale
img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Resize
img_gray = cv2.resize(img_gray, (IMG_SIZE, IMG_SIZE))

# Normalize
img_array = img_gray.astype("float32") / 255.0

# Add channel dimension (H, W, 1)
img_array = np.expand_dims(img_array, axis=-1)

# Add batch dimension (1, H, W, 1)
img_array = np.expand_dims(img_array, axis=0)

# Predict
preds = cnn_model.predict(img_array, verbose=0)[0]

pred_index = np.argmax(preds)
predicted_label = class_names[pred_index]
confidence = preds[pred_index] * 100

# Show result
plt.imshow(img_gray, cmap="gray")
plt.axis("off")
plt.title(f"Predicted: {predicted_label}\nConfidence: {confidence:.2f}%")
plt.show()


# Compare Binary vs Multi-Class (Very Important Table)

| Case        | Model Output                     | How to get confidence |
| ----------- | -------------------------------- | --------------------- |
| Binary      | Single scalar `prob = P(class1)` | `prob` or `1-prob`    |
| Multi-class | Vector `[p0, p1, ..., pN]`       | `preds[pred_index]`   |

- Multiclass means categorical, sparse

| Model Type                     | Output Shape     | Use `preds[pred_index]`? |
| ------------------------------ | ---------------- | ------------------------ |
| Binary (sigmoid, 1 unit)       | `(1,)` or scalar | ❌ NO                     |
| Categorical (softmax, N units) | `(N,)`           | ✅ YES                    |


# Side-by-Side Comparison (Very Important)

| Aspect           | Grayscale CNN     | Color VGG16      |
| ---------------- | ----------------- | ---------------- |
| Crop from        | `gray[...]`       | `frame[...]`     |
| Channels         | 1                 | 3                |
| Input shape      | `(H, W, 1)`       | `(H, W, 3)`      |
| Color conversion | ❌ Not needed      | ✅ Required       |
| Why              | Already grayscale | BGR → RGB needed |


## If you train your custom CNN on colorful (RGB) images, then in webcam inference you MUST use:

```
face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

In [ ]:
# For colorful images webcam code

import cv2
import numpy as np
import tensorflow as tf

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load trained model
IMG_SIZE = 224
MODEL_PATH = "emotion_vgg16_sparse_model.h5"

model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Emotion model loaded")

# Class names (must match training order!)
class_names = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(80, 80)
    )

    for (x, y, w, h) in faces:

        face = frame[y:y+h, x:x+w]  # color crop (BGR)
        if face.size == 0:
            continue

        # Preprocess
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)  # (H, W, 3)
        face = face.astype("float32") / 255.0
        face = np.expand_dims(face, axis=0)  # (1, H, W, 3)

        # Predict
        preds = model.predict(face, verbose=0)[0]
        pred_index = np.argmax(preds)
        confidence = preds[pred_index] * 100
        label = class_names[pred_index]

        text = f"{label} ({confidence:.2f}%)"

        # Draw
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(
            frame,
            text,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0,255,0),
            2
        )

    cv2.imshow("Emotion Detection (VGG16 + Sparse)", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# For color ful images using preprocessor_input

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input


# Load Haar Cascade
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load trained model
IMG_SIZE = 224
MODEL_PATH = "emotion_vgg16_sparse_model.h5"

model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Emotion model loaded")

# Class names (must match training order!)
class_names = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(80, 80)
    )

    for (x, y, w, h) in faces:

        face = frame[y:y+h, x:x+w]  # color crop (BGR)
        if face.size == 0:
            continue

        # Preprocess
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)   # (H, W, 3)

        # Add batch dimension first
        face = np.expand_dims(face, axis=0)

        # 🔥 VGG16 preprocessing (REPLACES /255)
        face = preprocess_input(face)


        # Predict
        preds = model.predict(face, verbose=0)[0]
        pred_index = np.argmax(preds)
        confidence = preds[pred_index] * 100
        label = class_names[pred_index]

        text = f"{label} ({confidence:.2f}%)"

        # Draw
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(
            frame,
            text,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0,255,0),
            2
        )

    cv2.imshow("Emotion Detection (VGG16 + Sparse)", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# For grey scale images webcam code

# WEBCAM CODE — CUSTOM CNN (GRAYSCALE, SPARSE)

import cv2
import numpy as np
import tensorflow as tf

# ==========================
# Load Haar Cascade
# ==========================
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# ==========================
# Load Trained CNN Model
# ==========================
IMG_SIZE = 48   # must match training
MODEL_PATH = "emotion_cnn_sparse_model.h5"

model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Grayscale CNN emotion model loaded")

# ==========================
# Class Names (MUST match training order)
# ==========================
class_names = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

# ==========================
# Start Webcam
# ==========================
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert full frame to grayscale (for face detection + model)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces on grayscale frame
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(80, 80)
    )

    for (x, y, w, h) in faces:

        # --------------------
        # Crop face (grayscale)
        # --------------------
        face_gray = gray[y:y+h, x:x+w]

        if face_gray.size == 0:
            continue

        # --------------------
        # Preprocess for CNN
        # --------------------
        face_gray = cv2.resize(face_gray, (IMG_SIZE, IMG_SIZE))

        # Normalize
        face_array = face_gray.astype("float32") / 255.0

        # Add channel dimension -> (H, W, 1)
        face_array = np.expand_dims(face_array, axis=-1)

        # Add batch dimension -> (1, H, W, 1)
        face_array = np.expand_dims(face_array, axis=0)

        # --------------------
        # Predict Emotion
        # --------------------
        preds = model.predict(face_array, verbose=0)[0]
        pred_index = np.argmax(preds)
        confidence = preds[pred_index] * 100
        label = class_names[pred_index]

        text = f"{label} ({confidence:.2f}%)"

        # --------------------
        # Draw Results
        # --------------------
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.putText(
            frame,
            text,
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2
        )

    cv2.imshow("Emotion Detection (CNN Grayscale)", frame)

    # ESC to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Extra

# from tensorflow.keras.applications.efficientnet import preprocess_input
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train_gen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True
# )

# val_gen = ImageDataGenerator(
#     preprocessing_function=preprocess_input
# )

# train_data = train_gen.flow_from_directory(
#     train_path,
#     target_size=(224,224),
#     batch_size=16,
#     class_mode='categorical',
#     color_mode='rgb',
#     shuffle=True
# )

# val_data = val_gen.flow_from_directory(
#     validation_path,
#     target_size=(224,224),
#     batch_size=16,
#     class_mode='categorical',
#     color_mode='rgb',
#     shuffle=False
# )